Funcoes utilizadas
Criar uma lista de comprimentos de strings em cada célula da coluna especificada
comprimentos = [len(str(celula)) for celula in df.iloc[: ,4]]
Encontrar o índice da célula com o maior número de caracteres
indice_maior = comprimentos.index(max(comprimentos))
Extrair a célula com o maior número de caracteres
celula_maior = df.iloc[indice_maior][4] df.iloc[: , 4][indice_maior] df.iloc[: , 4].value_counts() celula_maior df.iloc[:,0]
index = df.columns.get_loc(‘abcz’)
index df

IDANIMAL = CHAVE UNICA
MP = CHAVE UNICA -  Mes parto
AP = CHAVE UNICA - -Ano parto

LTOT = lactacao total
L365 = lactacao em ate 365 dias
L305 = lactacao em ate 305 dias
DE = Data de Secagem
ME = Mes de secagem
AE = ANO de Secagem
caso o LTOT, L365 e L305, DE, ME e AE sejam iguais ao arquivo old, pode ser descartado

In [50]:
import pandas as pd
import warnings
import re
import tables
warnings.filterwarnings('ignore')

## conectando no banco de dados

In [51]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

## Retornando toda tabela genealogia_embrapa e genealogia_abcz e tabela criadores do banco de dados para dataframes

In [52]:
sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
sql1 = "SELECT * FROM genealogia_abcz"
sql2 = "SELECT * FROM criadores"
sql3 = "SELECT * FROM producao_embrapa"
df_genealogia_embrapa = pd.read_sql(sql, mydb)
df_genealogia_embrapa=df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = pd.read_sql(sql1, mydb)
df_criadores = pd.read_sql(sql2, mydb)
df_producao = pd.read_sql(sql3, mydb)


## Lendo planilhas

In [53]:
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [54]:
#pedigree

In [55]:
#dados_2022

In [56]:
#dados_2023

In [57]:
#df_genealogia_embrapa

In [58]:
#df_genealogia_abcz

In [59]:
#df_criadores

In [60]:
#df_producao

## Criando um dataframe que é a junção dos dois dataframes, dados_2022 e dados_2023, pelas celulas que possuem os mesmos valores nas colunas IDANIMAL, AP, MP, L305, LTOT, L365, ME e AE.

In [61]:
df_dados_sem_atualizacao_entre_2022_2023 = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop(df_dados_sem_atualizacao_entre_2022_2023.filter(regex='_drop$').columns, axis=1)
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop_duplicates()
#df_dados_sem_atualizacao_entre_2022_2023

## Criando um dataframe com somente os novos dados inseridos em 2023
Esses são os dados que precisam ser inseridos no banco de dados

In [62]:
# faz o merge entre os dois dataframes, o dados com os dados repetidos
df_merge = pd.merge(dados_2023, df_dados_sem_atualizacao_entre_2022_2023, how='left', indicator=True, suffixes=('', '_drop'))

# filtra somente as linhas que estão no dataframe da esquerda
df_dados_novos_para_insercao_no_banco_de_dados = df_merge[df_merge['_merge'] == 'left_only']

# remove a coluna "_merge"
df_dados_novos_para_insercao_no_banco_de_dados.drop('_merge', axis=1, inplace=True)

df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_merge.filter(regex='_drop$').columns, axis=1)
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando coluna GS, oriunda do dataframe Pedigree, no dataframe df_dados_novos_para_insercao_no_banco_de_dados
inclusão necessária para localizar o animal

In [63]:
df_temporaria = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria
df_dados_novos_para_insercao_no_banco_de_dados

nome_antigo = 'SEXO'
nome_novo = 'sexo'
indice_coluna = df_dados_novos_para_insercao_no_banco_de_dados.columns.get_loc(nome_antigo)
df_dados_novos_para_insercao_no_banco_de_dados.rename(columns={nome_antigo: nome_novo}, inplace=True)
#df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela com somente as linhas que possuem gs = 0 no dataframe df_dados_novos_para_insercao_no_banco_de_dados

In [64]:
sub_tabela_gs_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['gs'] == 0]
#sub_tabela_gs_0

# Adicionando coluna cod_prod, cod_pai e cod_mae, oriundos do dataframe df_genealogia_abcz, na subtabela gs_0 dos animais que sao gs=0

In [65]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
df_temporaria1 = pd.merge(sub_tabela_gs_0, df_genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
sub_tabela_gs_0=df_temporaria1
sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()
#sub_tabela_gs_0

# Adicionando codprod, cod_pai, cod_mae no dataframe df_dados_novos_para_insercao_no_banco_de_dados a partir do dataframe sub_tabela_gs_0

In [66]:
df_temporaria4 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria4
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados



# Criando subtabela com os animais em que o gs da mae = 11 e que o rgmae comece com 3 letras

In [67]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSMAE'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_mae_0
# selecionar apenas as linhas que começam com letras
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando o cod_prod das maes no dataframe genealogia_abcz

In [68]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
sub_tabela_cod_mae_0 = pd.merge(sub_tabela_cod_mae_0, df_genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop(sub_tabela_cod_mae_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['_merge'] == 'both'].drop(columns='_merge')
#sub_tabela_cod_mae_0


# Adicionando o cod_prod_mae_gen das maes no dataframe df_dados_novos

In [69]:
df_temporaria5 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_mae_0[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria4
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria5 = df_temporaria5.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados


# Criando subtabela em que o gs do pai = 11 e que os 3 primeiros caracteres do rgpai sao letras

In [70]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSPAI'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_pai_0
# selecionar apenas as linhas que começam com letras
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando o cod_prod dos pais no dataframe genealogia_abcz

In [71]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
sub_tabela_cod_pai_0 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop(sub_tabela_cod_pai_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['_merge'] == 'both'].drop(columns='_merge')
#sub_tabela_cod_pai_0

# Adicionando o cod_prod_pai_gen dos pais no dataframe df_dados_novos

In [72]:
df_temporaria6 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
df_temporaria6 = df_temporaria6.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria6
#df_dados_novos_para_insercao_no_banco_de_dados

# Pegando valores das colunas cod_prod_pai_gen e cod_prod_mae_gen e atualizando colunas  cod_pai e cod_mae no DF_DADOS_NOVOS

In [73]:
df_temporaria6 = df_temporaria5
df_dados_novos_para_insercao_no_banco_de_dados['cod_pai'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_pai_gen'], inplace=True)
df_dados_novos_para_insercao_no_banco_de_dados['cod_mae'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_mae_gen'], inplace=True)
#Dando drop nas colunas antigas
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_pai_gen')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_mae_gen')
#df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela em que o gs do pai = 11 mas os 3 primeiros caracteres do rgpai nao sao letras

In [74]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSPAI'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_pai_0
# selecionar apenas as linhas que NÃO começam com letras
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[~sub_tabela_cod_pai_0['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando cod_prod_pai no genealogia_abcz dos animais em que o registro nao comeca com 3 letras

In [75]:
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
df_genealogia_abcz = df_genealogia_abcz.drop_duplicates()
#criando coluna com os 10 caracteres do nome
sub_tabela_cod_pai_0['nome11caracteres'] = sub_tabela_cod_pai_0['nomep'].str.slice(start=0, stop=11)
df_genealogia_abcz['nome11caracteres'] = df_genealogia_abcz['nome'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
#sub_tabela_cod_pai_0['nome10caracteres'] = sub_tabela_cod_pai_0['nome10caracteres'].str.replace('\s+', '', regex=True)
#df_genealogia_abcz['nome10caracteres'] = df_genealogia_abcz['nome10caracteres'].str.replace('\s+', '', regex=True)

sub_tabela_cod_pai_0 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'nome11caracteres','cod_prod_pai_gen']], left_on=['RGPAI', 'nome11caracteres'], right_on=['RGVACA', 'nome11caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop(sub_tabela_cod_pai_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['_merge'] == 'both'].drop(columns='_merge')
#sub_tabela_cod_pai_0

# Adicionando o cod_prod_pai_gen dos pais no dataframe df_dados_novos
obs: dos REGPAI que nao comecam com 3 caracteres

In [76]:
df_temporaria7 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria7 = df_temporaria7.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria7
#df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela em que o gs da mae = 11 mas os 3 primeiros caracteres do rgmae nao sao letras

In [77]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSMAE'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_mae_0
# selecionar apenas as linhas que NÃO começam com letras
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[~sub_tabela_cod_mae_0['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando cod_prod_mae no genealogia abcz dos animais em que o registro nao comeca com 3 letras

In [78]:
sub_tabela_cod_mae_0['nome11caracteres'] = sub_tabela_cod_mae_0['nomem'].str.slice(start=0, stop=11)
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
#criando coluna com os 10 caracteres do nome
sub_tabela_cod_mae_0 = pd.merge(sub_tabela_cod_mae_0, df_genealogia_abcz[['RGVACA','nome11caracteres' ,'cod_prod_mae_gen']], left_on=['RGMAE', 'nome11caracteres'], right_on=['RGVACA', 'nome11caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop(sub_tabela_cod_mae_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['_merge'] == 'both'].drop(columns='_merge')
#sub_tabela_cod_mae_0


# Adicionando cod_prod_mae_gen no df_dados_novos
Obs: regmae com menos de 3 caracteres

In [79]:
df_temporaria8 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_mae_0[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
df_temporaria8 = df_temporaria8.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria8
#df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando os dados do cod_prod_mae_gen e do cod_prod_pai_gen no cod_pai e cod_mae
OBS: dados com o regpai e regmae com < 3 letras

In [80]:
df_dados_novos_para_insercao_no_banco_de_dados['cod_pai'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_pai_gen'], inplace=True)
df_dados_novos_para_insercao_no_banco_de_dados['cod_mae'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_mae_gen'], inplace=True)
#Dando drop nas colunas antigas
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_pai_gen')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_mae_gen')
#df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando reb no dataframe df_dados_novos_para_insercao_no_banco_de_dados
Obs: Cod_gil esta na tabela de criadores

In [81]:
df_criadores = df_criadores.rename(columns={'codgiro':'reb_giro'})
df_dados_novos_para_insercao_no_banco_de_dados = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, df_criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()

In [82]:
#verifica_reb = df_criadores.groupby('reb_giro')['reb'].nunique()
#verifica_reb

## Calculando a diferença entre a data dos novos partos com as ultimas datas presentes na tabela de dados_2022 para validar se ha alguma inconsistência

obs: Caso a diferença seja menor que 250 dias é provável que haja alguma falha.

In [83]:

df_diferenca_entre_partos_novos_e_do_banco = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on=['IDANIMAL'], suffixes=('', '_drop'))
df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop_duplicates()
#df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop(df_diferenca_entre_partos_novos_e_do_banco.filter(regex='_drop$').columns, axis=1)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].astype(str)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].apply(pd.to_datetime)
df_diferenca_entre_partos_novos_e_do_banco['diferenca_em_dias'] = (df_diferenca_entre_partos_novos_e_do_banco['parto'] - df_diferenca_entre_partos_novos_e_do_banco['parto_drop']).dt.days
#df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop', 'diferenca_em_dias']]

## Pegando da tabela de animais que serão inseridos no banco aqueles que ja possuem IDANIMAL cadastrado
Comparado ao dataframe  dados_2022

In [84]:
df_animais_com_ID = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on =['IDANIMAL'], suffixes=('', '_drop'))
df_animais_com_ID = df_animais_com_ID.drop(df_animais_com_ID.filter(regex='_drop$').columns, axis=1)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2,2,60292021122,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,3,2,74502020132,NaN,6,F,NaN,NaN,NaN,7450-GIRO
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2,2,33042020122,NaN,4,F,NaN,3287666.0,NaN,3304-GIRO
4,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,2,2,55692021222,NaN,4,F,NaN,9265878.0,NaN,5569-GIRO
5,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,2,2,55692021222,NaN,3,F,NaN,2225848.0,NaN,5569-GIRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,1129,MORADIA ANGICO,7,9556-F,VALE OURO DE BRASILIA,11,A6796,CABANA ANGICO,78,1606,...,2,2,11292021222,NaN,4,F,NaN,1351076.0,NaN,52721
681,2381,CLASSICA OCIDENTE JM NOVO HORI,14,5193-AE,OCIDENTE BEM FEITOR CAL,11,CAL4605,CINDERELA MODELO JM NOVO HORIZ,12,AF-2652,...,2,2,23812020122,NaN,2,F,NaN,3287666.0,NaN,31300-ABCZ
684,5569,GABI FIV VALE OURO EVD,14,4417-AT,VALE OURO TE SILVANIA,11,EFC464,LAMBADA DRAMATIC SANTA LUZIA,12,AG-0944,...,2,2,55692020122,NaN,2,F,NaN,3890526.0,NaN,5569-GIRO
685,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,2,60292021222,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ


## Pegando da tabela de animais que serão inseridos no banco, todos aqueles que não possuem IDANIMAL cadastrado
comparado ao dataframe dados_2022

In [85]:
df_animais_sem_id = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on='IDANIMAL', how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['_merge'] == 'left_only'].drop(columns='_merge')
#df_animais_sem_id

In [86]:
#dados_2022[dados_2022['IDANIMAL'] == 1631099]
#df_verificando_animais_com_id_na_genealogia_embrapa['IDANIMAL'].value_counts()
#verifica_reb = df_verificando_animais_com_id_na_genealogia_embrapa.groupby('IDANIMAL')['id_produto'].nunique()
#verifica_reb

# Adicionando a coluna id_produto ao dataframe df_animais_com_ID

In [87]:
df_genealogia_embrapa['IDANIMAL'] = df_genealogia_embrapa['IDANIMAL'].astype('int')
df_animais_com_ID = pd.merge(df_animais_com_ID, df_genealogia_embrapa[['IDANIMAL', 'id_produto']], on='IDANIMAL', how='left', indicator=True)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID = df_animais_com_ID.drop(columns='_merge')
#df_animais_com_ID

## Lista de animais do dataframe sem ID em relacao a dados_2022 e que possuem ID_animal no dataframe da genealogia_Embrapa
Obs: Tem que ter ID_produto

In [88]:
df_animais_sem_id = df_animais_sem_id.add_suffix('_esq')
df_genealogia_embrapa = df_genealogia_embrapa.add_suffix('_dir')

df_animais_sem_id_2022_que_possuem_id_no_banco = pd.merge(df_animais_sem_id, df_genealogia_embrapa, left_on=['IDANIMAL_esq'], right_on=['IDANIMAL_dir'], how='left', indicator=True)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop_duplicates()
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco[df_animais_sem_id_2022_que_possuem_id_no_banco['_merge'] == 'both']

# dropa as colunas que não são necessárias
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_dir').columns.difference(['id_produto_dir']), axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
#df_animais_sem_id_2022_que_possuem_id_no_banco

# Atualizando lista de animais que possuem ID encontrado

In [89]:
# Juntando os dataframes
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_sem_id_2022_que_possuem_id_no_banco])
#df_animais_com_ID

# Atualizando lista de animais sem ID

In [90]:
#Tirando o sufixo da tabela
df_animais_sem_id = df_animais_sem_id.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
# Imprimir o resultado
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto'].isnull()]
df_animais_sem_id = df_animais_sem_id.drop(columns='_merge')
#df_animais_sem_id

# Fazendo busca no dataframe producao para encontrar id_produto dos animais sem id
conferir manualmente se entram ou não no banco

In [91]:
#Criando coluna com o ano de parto e mes parto no dataframe df_producao
df_producao['parto'] = df_producao['parto'].astype(str)
df_producao['AP'] = df_producao['parto'].str.slice(start=0, stop=4)
df_producao['MP'] = df_producao['parto'].str.slice(start=5, stop=7)
#Pegando os 10 primeiros caracteres de nome dos dataframes aniamis sem id e producao
df_producao['nome10caracteres'] = df_producao['nome'].str.slice(start=0, stop=11)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nomea'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
df_producao['nome10caracteres'] = df_producao['nome10caracteres'].str.replace('\s+', '', regex=True)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nome10caracteres'].str.replace('\s+', '', regex=True)

df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_producao = df_producao.drop(df_producao.filter(regex='_drop$').columns, axis=1)

df_producao['AP'] = df_producao['AP'].astype(int)
df_producao['MP'] = df_producao['MP'].astype(int)
df_animais_sem_id['AP'] = df_animais_sem_id['AP'].astype(int)
df_animais_sem_id['MP'] = df_animais_sem_id['MP'].astype(int)
df_animais_com_id_na_producao = pd.merge(df_animais_sem_id, df_producao[['reb','AP', 'MP', 'nome10caracteres','id_produto']], on=['reb', 'AP', 'MP', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_producao = df_animais_com_id_na_producao[df_animais_com_id_na_producao['_merge'] == 'both']
df_animais_com_id_na_producao = df_animais_com_id_na_producao.drop(columns={'_merge', 'id_produto'})
df_animais_com_id_na_producao = df_animais_com_id_na_producao.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_producao

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,nome10caracteres,id_produto
216,11818,BACANA CABO VERDE,11,JCVL2511,GABEIRA GIROESTE,11,LUF182,VISCAYA CABO VERDE,11,JCVL1482,...,118182021232,NaN,0,F,16530248.0,9518703.0,13499194.0,39977-ABCZ,BACANACABO,JCVL2511
388,2381,GALENA FABULOSO JM NOVO HORIZO,38,7020-BC,FABULOSO DO BASA,11,BASP63,ALVORADA 122 JM NOVO HORIZONTE,34,2686-AG,...,23812021222,NaN,3,F,NaN,10796351.0,NaN,31300-ABCZ,GALENAFABU,216096
459,2381,FANTASIA FABULOSO JM NOVO HORI,38,4896-AZ,FABULOSO DO BASA,11,BASP63,NOVIAA I FIV GOLDEN JM NOVO HO,34,3673-U,...,23812021222,NaN,3,F,NaN,10796351.0,NaN,31300-ABCZ,FANTASIAFA,216094


# Atualizando animais com id com os novos ids_produtos encontrados no dataframe de producao

In [92]:
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_com_id_na_producao])
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto,nome10caracteres
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,60292021122,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ,GIRO1383626,NaN
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,74502020132,NaN,6,F,NaN,NaN,NaN,7450-GIRO,GIRO1608903,NaN
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,33042020122,NaN,4,F,NaN,3287666.0,NaN,3304-GIRO,GIRO1215331,NaN
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,55692021222,NaN,4,F,NaN,9265878.0,NaN,5569-GIRO,GIRO2022656,NaN
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,55692021222,NaN,3,F,NaN,2225848.0,NaN,5569-GIRO,GIRO1653303,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,6029,GRUTA FR RECREIO,12,4847-BG,0,0,0,0,0,0,...,60292021222,NaN,4,F,NaN,NaN,NaN,31365-ABCZ,214852,NaN
591,1945,ABAMA 2 FIV M. VERDE,11,ISPG1286,C.A.SANSAO,11,KCA472,ABAMA TE DE KUBERA,11,ACFG80,...,19452021222,NaN,0,F,11869706.0,2730835.0,3332560.0,37634-ABCZ,ISPG1286,NaN
216,11818,BACANA CABO VERDE,11,JCVL2511,GABEIRA GIROESTE,11,LUF182,VISCAYA CABO VERDE,11,JCVL1482,...,118182021232,NaN,0,F,16530248.0,9518703.0,13499194.0,39977-ABCZ,JCVL2511,BACANACABO
388,2381,GALENA FABULOSO JM NOVO HORIZO,38,7020-BC,FABULOSO DO BASA,11,BASP63,ALVORADA 122 JM NOVO HORIZONTE,34,2686-AG,...,23812021222,NaN,3,F,NaN,10796351.0,NaN,31300-ABCZ,216096,GALENAFABU


# Atualizando lista de animais sem id

In [93]:
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto_drop'].isnull()]
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns='_merge')
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto,nome10caracteres
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,74612019122,NaN,5,F,NaN,NaN,NaN,1291,NaN,NANAIMPERA
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,106392021122,NaN,4,F,NaN,NaN,9513273.0,10639-GIRO,NaN,LANAAFIV
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN,NaN,EDILAINEIM
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,19452021222,NaN,4,F,NaN,NaN,10621711.0,37634-ABCZ,NaN,ARMAMERIDI
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN,NaN,CELESTE136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,19452022222,NaN,4,F,NaN,NaN,11428022.0,37634-ABCZ,NaN,JPZQUIRERA
579,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,23812021222,NaN,4,F,NaN,NaN,12866148.0,31300-ABCZ,NaN,DALIAIFIV
580,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,1322019222,NaN,6,F,NaN,NaN,NaN,31835-ABCZ,NaN,QUINTAICO
581,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,69152021222,NaN,5,F,NaN,NaN,NaN,27729-ABCZ,NaN,LIADASARA


# Criando subtabela dos animais sem id em que o comprac = 11 e o rgvaca comeca com 3 letras

In [94]:
df_animais_com_id_na_genealogia0 = df_animais_sem_id[df_animais_sem_id['COMPRAC']==11]
# selecionar apenas as linhas que começam com letras
df_animais_com_id_na_genealogia0 = df_animais_com_id_na_genealogia0[df_animais_com_id_na_genealogia0['RGVACA'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_animais_com_id_na_genealogia0

# Criando subtabela da df_genealogia_embrapa em que o gs= 0

In [95]:
#Tirando o sufixo da tabela
df_genealogia_embrapa = df_genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
df_subtabela_genealogia_emb = df_genealogia_embrapa[df_genealogia_embrapa['gs']==0]
#df_subtabela_genealogia_emb

# Procurando animais na genealogia embrapa atraves da subtabela

In [96]:
df_animais_encontrados_subtabela_genealogia = pd.merge(df_animais_com_id_na_genealogia0,df_subtabela_genealogia_emb[['id_produto', 'registro']], left_on=['RGVACA'], right_on=['registro'], how='left', indicator=True)
df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia[df_animais_encontrados_subtabela_genealogia['_merge'] == 'both']
df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop(columns='_merge')
df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop(columns='registro')
df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop_duplicates()
df_animais_encontrados_subtabela_genealogia

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto_x,nome10caracteres,id_produto_y
2,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,NaN,0,F,17054525.0,4176523.0,13961623.0,5192-GIRO,NaN,URUPIANGAF,RRP8772
3,5192,PRENDA FIV DE BRAS.,11,RRP8191,GENGIS KHAN DE BRASILIA,11,RRP6097,FERVURA FIV DE BRASILIA,11,RRP5998,...,NaN,0,F,14926361.0,9265878.0,8720772.0,5192-GIRO,NaN,PRENDAFIV,RRP8191
5,22541,FELICE FIV CABO VERDE,11,JCVL3334,JAGUAR TE DO GAVIAO,11,GAV291,ZILAH FIV CABO VERDE,11,JCVL1624,...,NaN,0,F,17198554.0,4176523.0,13865301.0,NaN,NaN,FELICEFIV,JCVL3334
7,42583,BARILOCHE KENYO,11,KOK844,ELDORADO FIV KENYO,11,KOK96,IZABELLE FIV KENYO,11,KOK526,...,NaN,0,F,14672175.0,9451056.0,12324993.0,NaN,NaN,BARILOCHEK,KOK844
9,5192,RAGIN FIV DE BRAS.,11,RRP8395,JAGUAR TE DO GAVIAO,11,GAV291,ISCA DE BRASILIA,11,RRP6699,...,NaN,0,F,15421321.0,4176523.0,11066847.0,5192-GIRO,NaN,RAGINFIVD,RRP8395
10,5192,RUANA DE BRAS.,11,RRP8430,BACO DA RTPA,11,NLT130,LALA DE BRAS.,11,RRP7119,...,NaN,0,F,15438291.0,12480405.0,12161575.0,5192-GIRO,NaN,RUANADEBR,RRP8430
11,5192,LIZ TEATRO,11,RARS10,TEATRO DA SILVANIA,11,EFC383,RENUNCIA FIV DE BRAS.,11,RRP8249,...,NaN,0,F,17112475.0,3540404.0,15128061.0,5192-GIRO,NaN,LIZTEATRO,RARS10
12,42583,MILENA KENYO,11,KOK846,NOBRE TE CAL,11,CAL4397,JARANA FIV KENYO,11,KOK625,...,NaN,0,F,14673091.0,2989503.0,12894710.0,NaN,NaN,MILENAKENY,KOK846
17,5192,REDE FIV DE BRAS.,11,RRP8488,GENGIS KHAN DE BRASILIA,11,RRP6097,DELI DE BRAS.,11,RRP5562,...,NaN,0,F,15617034.0,9265878.0,6293655.0,5192-GIRO,NaN,REDEFIVDE,RRP8488
18,5192,RENUNCIA FIV DE BRAS.,11,RRP8249,FARDO FIV F.MUTUM,11,MUT697,ESFERA DE BRASILIA TN2 DE BRAS,11,RRP7530,...,NaN,0,F,15128061.0,8530023.0,13305655.0,5192-GIRO,NaN,RENUNCIAFI,RRP8249


# Atualizando lista de animais com id

In [97]:
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_encontrados_subtabela_genealogia])

df_animais_com_ID['id_produto'].fillna(df_animais_com_ID['id_produto_x'], inplace=True)
df_animais_com_ID['id_produto'].fillna(df_animais_com_ID['id_produto_y'], inplace=True)
df_animais_com_ID = df_animais_com_ID.drop(columns='id_produto_x')
df_animais_com_ID = df_animais_com_ID.drop(columns='id_produto_y')
df_animais_com_ID = df_animais_com_ID.drop_duplicates()

df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto,nome10caracteres
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,60292021122,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ,GIRO1383626,NaN
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,74502020132,NaN,6,F,NaN,NaN,NaN,7450-GIRO,GIRO1608903,NaN
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,33042020122,NaN,4,F,NaN,3287666.0,NaN,3304-GIRO,GIRO1215331,NaN
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,55692021222,NaN,4,F,NaN,9265878.0,NaN,5569-GIRO,GIRO2022656,NaN
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,55692021222,NaN,3,F,NaN,2225848.0,NaN,5569-GIRO,GIRO1653303,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,5192,REDE FIV DE BRAS.,11,RRP8488,GENGIS KHAN DE BRASILIA,11,RRP6097,DELI DE BRAS.,11,RRP5562,...,51922020132,NaN,0,F,15617034.0,9265878.0,6293655.0,5192-GIRO,RRP8488,REDEFIVDE
18,5192,RENUNCIA FIV DE BRAS.,11,RRP8249,FARDO FIV F.MUTUM,11,MUT697,ESFERA DE BRASILIA TN2 DE BRAS,11,RRP7530,...,51922020132,NaN,0,F,15128061.0,8530023.0,13305655.0,5192-GIRO,RRP8249,RENUNCIAFI
19,5192,ROTINA FIV DE BRAS.,11,RRP8273,FARDO FIV F.MUTUM,11,MUT697,ILDA FIV DE BRAS.,11,RRP6695,...,51922020132,NaN,0,F,15163671.0,8530023.0,11066844.0,5192-GIRO,RRP8273,ROTINAFIV
24,5192,PILLAR FIV DE BRAS.,11,RRP8130,DIAMANTE TE DE BRASILIA,11,RRP5640,JAMARIA FIV DE BRASILIA,11,RRP6958,...,51922021132,NaN,0,F,14820082.0,6716196.0,11600828.0,5192-GIRO,RRP8130,PILLARFIV


# Atualizando lista de animais sem id

In [98]:
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_encontrados_subtabela_genealogia, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto_y'].isnull()]
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns='_merge')
df_animais_sem_id = df_animais_sem_id.drop_duplicates()
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
#df_animais_sem_ID = df_animais_com_ID.drop(columns='id_produto_x', axis =1)
#df_animais_sem_ID = df_animais_com_ID.drop(columns='id_produto_y', axis =1)
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto,nome10caracteres,id_produto_x,id_produto_y
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,5,F,NaN,NaN,NaN,1291,NaN,NANAIMPERA,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,4,F,NaN,NaN,9513273.0,10639-GIRO,NaN,LANAAFIV,NaN,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,4,F,NaN,4320031.0,NaN,NaN,NaN,EDILAINEIM,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,4,F,NaN,NaN,10621711.0,37634-ABCZ,NaN,ARMAMERIDI,NaN,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,4,F,NaN,3540404.0,NaN,NaN,NaN,CELESTE136,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,4,F,NaN,NaN,11428022.0,37634-ABCZ,NaN,JPZQUIRERA,NaN,NaN
576,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,4,F,NaN,NaN,12866148.0,31300-ABCZ,NaN,DALIAIFIV,NaN,NaN
577,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,6,F,NaN,NaN,NaN,31835-ABCZ,NaN,QUINTAICO,NaN,NaN
578,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,5,F,NaN,NaN,NaN,27729-ABCZ,NaN,LIADASARA,NaN,NaN


# Fazendo a busca na tabela genealogia_embrapa
Fazer verificacao manual

In [99]:
#Retirando sufixo tabela genealogia
df_genealogia_embrapa = df_genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
#Pegando ano nascimento nas colunas
df_genealogia_embrapa['nasc_est'] = df_genealogia_embrapa['nasc_est'].astype(str)
df_genealogia_embrapa['AN'] = df_genealogia_embrapa['nasc_est'].str.slice(start=0, stop=4)
df_animais_sem_id['nasc'] = df_animais_sem_id['nasc'].astype(str)
df_animais_sem_id['AN'] = df_animais_sem_id['nasc'].str.slice(start=0, stop=4)
#Criando coluna com os 10 caracteres do nome na genealogia_embrapa
df_genealogia_embrapa['nome10caracteres'] = df_genealogia_embrapa['nome'].str.slice(start=0, stop=11)
#Convertendo coluna ano e mes para tipo inteiro
#df_genealogia_embrapa['AN'] = df_genealogia_embrapa['AN'].astype(int)
#Limpando tabelas
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_genealogia_embrapa = df_genealogia_embrapa.drop(df_genealogia_embrapa.filter(regex='_drop$').columns, axis=1)
#criando dataframe com os ids_encontrados
df_animais_com_id_na_genealogia = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs','RGMAE', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia[df_animais_com_id_na_genealogia['_merge'] == 'both']
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(columns={'_merge', 'id_produto', 'registro_mae', 'registro_pai'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(df_animais_com_id_na_genealogia.filter(regex='_drop$').columns, axis=1)
df_animais_com_id_na_genealogia

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,sexo,cod_prod,cod_pai,cod_mae,reb,nome10caracteres,id_produto_x,id_produto_y,AN,id_produto


In [100]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,cod_pai,cod_mae,reb,id_produto,nome10caracteres,id_produto_x,id_produto_y,AN,id_produto_drop,_merge
401,1945,BAILARINA,11,MBLO12,GENGIS KHAN DE BRASILIA,11,RRP6097,BEL DA RTPA,11,NLT89,...,9265878.0,12335487.0,37634-ABCZ,NaN,BAILARINA,NaN,NaN,2020,RGIR34,both
406,1945,FANTASIA,11,MBLO18,JAGUAR TE DO GAVIAO,11,GAV291,ELECTRA FIV PEDREIRA,11,PPG41,...,4176523.0,10767949.0,37634-ABCZ,NaN,FANTASIA,NaN,NaN,2020,TAN219,both


# Salvando df_animais_sem_id

In [101]:
df_animais_sem_id.to_hdf('df_animais_sem_id.h5', key='df', mode='w')
df_genealogia_embrapa.to_hdf('df_genealogia_embrapa.h5', key='df', mode='w')

In [102]:
df_genealogia_embrapa.to_hdf('df_genealogia_embrapa.h5', key='df', mode='w')